# **ETL PROCESS**

## Description

This notebook performs an ETL (Extract, Transform, Load) process on US air pollution data from 2000 to 2016. It imports the raw CSV, cleans the data by removing missing values and unnecessary columns, renames columns for clarity, and prepares a sample for analysis. The final processed data is saved for further use in analytics or machine learning.

## Objectives

* Fetch data and save it as raw data file and upload it to the workspace. Take the data through the ETL process to clean it.

## Inputs

* Raw CSV data file.

## Outputs

* This notebook will hope to generate a clean CSV file of the data. 

## Additional Comments

* This dataset was sourced from Kaggle and contains data regarding air pollution quality in the US in from 2000 to 2016.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\F_bee\\Documents\\vs-code\\vs-code-projects\\air-quality-dashboard\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\F_bee\\Documents\\vs-code\\vs-code-projects\\air-quality-dashboard'

# Section 1

Extract and read the data.

In [4]:
# Import all necessary packages
import numpy as np
import pandas as pd
import sklearn as sk
import feature_engine as fe 
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

print("All packages imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sk.__version__}")
print(f"Feature-engine version: {fe.__version__}")

All packages imported successfully!
NumPy version: 1.26.1
Pandas version: 2.1.1
Scikit-learn version: 1.3.1
Feature-engine version: 1.6.1


In [5]:
# Read data and return full DataFrame with shape to make sure everything is working
# Also return the data shape
df = pd.read_csv("inputs/pollution_us_2000_2016.csv")
print("Data loaded successfully!")
print(f"DataFrame shape: {df.shape}")
df

Data loaded successfully!
DataFrame shape: (1746661, 29)


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.200,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.200,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.200,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.200,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.600,23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746656,24599,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,...,Parts per billion,0.000000,0.0,2,NaN,Parts per million,0.091667,0.100,2,1.0
1746657,24600,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.067714,0.127,0,NaN
1746658,24601,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.100000,0.100,0,1.0
1746659,24602,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,0.000000,0.0,5,NaN,Parts per million,0.067714,0.127,0,NaN


In [6]:
# Return the first five values of the DataFrame for future observation purposes where necessary
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [8]:
# Drop missing values from df1
df = df.dropna()
print(f"DataFrame shape: {df.shape}")
print("Missing values dropped from df.")
df.head()

DataFrame shape: (436876, 29)
Missing values dropped from df.


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
5,5,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,1.066667,2.3,0,26.0
9,9,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-03,Parts per billion,...,Parts per billion,5.250000,11.0,19,16.0,Parts per million,1.762500,2.5,8,28.0
13,13,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-04,Parts per billion,...,Parts per billion,7.083333,16.0,8,23.0,Parts per million,1.829167,3.0,23,34.0
17,17,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-05,Parts per billion,...,Parts per billion,8.708333,15.0,7,21.0,Parts per million,2.700000,3.7,2,42.0


In [39]:
# Extract a random, fractioned sample of the data of 2500 values for analytic purposes
# Also return the new data shape
df1 = df.sample(frac=0.005722, random_state=10)
print(f"DataFrame shape: {df1.shape}")
df1.head()

DataFrame shape: (2500, 29)


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
729196,99891,45,19,46,390 BULLS ISLAND ROAD (AWENDAW) [Cape Romain],South Carolina,Charleston,Not in a city,2007-10-22,Parts per billion,...,Parts per billion,0.000000,0.0,0,0.0,Parts per million,0.200000,0.2,0,2.0
156520,64874,36,81,97,56TH AVE AT SPRINGFIELD BLVD,New York,Queens,New York,2001-07-11,Parts per billion,...,Parts per billion,5.041667,18.0,11,26.0,Parts per million,0.429167,0.5,2,6.0
1473635,9223,5,119,7,PIKE AVE AT RIVER ROAD,Arkansas,Pulaski,North Little Rock,2014-07-28,Parts per billion,...,Parts per billion,0.652174,0.8,12,0.0,Parts per million,0.237500,0.3,0,3.0
1334075,4239,5,119,7,PIKE AVE AT RIVER ROAD,Arkansas,Pulaski,North Little Rock,2013-01-12,Parts per billion,...,Parts per billion,0.887500,1.2,0,1.0,Parts per million,1.083333,1.2,0,14.0
218156,37039,6,83,4003,"STS POWER PLANT, VANDENBERG AFB",California,Santa Barbara,Vandenberg Air Force Base,2002-01-27,Parts per billion,...,Parts per billion,0.130435,1.0,20,1.0,Parts per million,0.200000,0.2,0,2.0


---

# Section 2

Begin to transform the data, creating transformers and the pipeline code.

In [40]:
# Check the current columns
print("Data loaded successfully!")
print("Available columns:")
print(df.columns.tolist())

Data loaded successfully!
Available columns:
['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address', 'State', 'County', 'City', 'Date Local', 'NO2 Units', 'NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 Units', 'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units', 'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI', 'CO Units', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI']


In [41]:
# Check minimum values for numerical columns only
print("Minimum values for numerical columns:")
print(df1.select_dtypes(include="number").min())

Minimum values for numerical columns:
Unnamed: 0           29.000000
State Code            1.000000
County Code           1.000000
Site Num              1.000000
NO2 Mean             -0.114286
NO2 1st Max Value     0.000000
NO2 1st Max Hour      0.000000
NO2 AQI               0.000000
O3 Mean               0.000625
O3 1st Max Value      0.002000
O3 1st Max Hour       0.000000
O3 AQI                2.000000
SO2 Mean             -1.000000
SO2 1st Max Value    -1.000000
SO2 1st Max Hour      0.000000
SO2 AQI               0.000000
CO Mean               0.000000
CO 1st Max Value      0.000000
CO 1st Max Hour       0.000000
CO AQI                0.000000
dtype: float64


In [42]:
# Check maximum values for numerical columns only
print("Maximum values for numerical columns:")
print(df1.select_dtypes(include="number").max())

Maximum values for numerical columns:
Unnamed: 0           134289.000000
State Code               80.000000
County Code             650.000000
Site Num               9997.000000
NO2 Mean                 98.130435
NO2 1st Max Value       157.000000
NO2 1st Max Hour         23.000000
NO2 AQI                 112.000000
O3 Mean                   0.067333
O3 1st Max Value          0.104000
O3 1st Max Hour          23.000000
O3 AQI                  172.000000
SO2 Mean                 32.956522
SO2 1st Max Value       131.000000
SO2 1st Max Hour         23.000000
SO2 AQI                 126.000000
CO Mean                   3.491667
CO 1st Max Value          5.600000
CO 1st Max Hour          23.000000
CO AQI                   62.000000
dtype: float64


In [ ]:
print("Data loaded successfully!")
df1.duplicated().sum()

Data loaded successfully!


0

In [ ]:
print("Data loaded successfully!") 
df1.isnull().sum()

Data loaded successfully!


Unnamed: 0           0
State Code           0
County Code          0
Site Num             0
Address              0
State                0
County               0
City                 0
Date Local           0
NO2 Units            0
NO2 Mean             0
NO2 1st Max Value    0
NO2 1st Max Hour     0
NO2 AQI              0
O3 Units             0
O3 Mean              0
O3 1st Max Value     0
O3 1st Max Hour      0
O3 AQI               0
SO2 Units            0
SO2 Mean             0
SO2 1st Max Value    0
SO2 1st Max Hour     0
SO2 AQI              0
CO Units             0
CO Mean              0
CO 1st Max Value     0
CO 1st Max Hour      0
CO AQI               0
dtype: int64

In [45]:
# Print column names and their data types for df1
print("Column names:")
print(df1.columns.tolist())
print(f"DataFrame shape: {df1.shape}")
print(df1.dtypes)

Column names:
['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address', 'State', 'County', 'City', 'Date Local', 'NO2 Units', 'NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 Units', 'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units', 'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI', 'CO Units', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI']
DataFrame shape: (2500, 29)
Unnamed: 0             int64
State Code             int64
County Code            int64
Site Num               int64
Address               object
State                 object
County                object
City                  object
Date Local            object
NO2 Units             object
NO2 Mean             float64
NO2 1st Max Value    float64
NO2 1st Max Hour       int64
NO2 AQI                int64
O3 Units              object
O3 Mean              float64
O3 1st Max Value     float64
O3 1st Max Hour        int64
O3 AQI                 int64
S

In [46]:
# Drop columns with "Unnamed" in their name from df1
df1 = df1.loc[:, ~df1.columns.str.contains("^Unnamed")]
print("Columns with 'Unnamed' dropped.")

Columns with 'Unnamed' dropped.


In [47]:
# Rename "Site Num" column to "Site Number" in df1
df1 = df1.rename(columns={"Site Num": "Site Number"})
print("Column 'Site Num' renamed to 'Site Number'.")

Column 'Site Num' renamed to 'Site Number'.


---

# Section 3

Load the data to the necessary file.

In [49]:
# Save the processed datasets
df1.to_csv("outputs/analysis.csv", index=False)

---

# Section 4

Here are a few insights from this notebook:

* The air pollution dataset covers US data from 2000 to 2016 and includes multiple pollutants and site information.
* Data cleaning steps removed missing values and unnecessary columns, improving data quality for analysis.
* The column "Site Num" was renamed to "Site Number" for clarity.
* A random sample of 2,500 rows was extracted for efficient analysis.
* The cleaned data is ready for further statistical analysis, visualization, or machine learning tasks.

NOTE

* The dataset was very large so had to shorten it
* Also, working against time constraints found itself difficult, though the necessary data was manifested

---

# Conclusion

This ETL session was quite intriguing. Time was spent trying to deduce the right dataset to work with, but in the end the dataset was generated and loaded to further analyze.